# Hugging Face Transformers 微调训练入门

本示例将介绍基于 Transformers 实现模型微调训练的主要流程，包括：
- 数据集下载
- 数据预处理
- 训练超参数配置
- 训练评估指标设置
- 训练器基本介绍
- 实战训练
- 模型保存

## YelpReviewFull 数据集

**Hugging Face 数据集：[ YelpReviewFull ](https://huggingface.co/datasets/yelp_review_full)**

### 数据集摘要

Yelp评论数据集包括来自Yelp的评论。它是从Yelp Dataset Challenge 2015数据中提取的。

### 支持的任务和排行榜
文本分类、情感分类：该数据集主要用于文本分类：给定文本，预测情感。

### 语言
这些评论主要以英语编写。

### 数据集结构

#### 数据实例
一个典型的数据点包括文本和相应的标签。

来自YelpReviewFull测试集的示例如下：

```json
{
    'label': 0,
    'text': 'I got \'new\' tires from them and within two weeks got a flat. I took my car to a local mechanic to see if i could get the hole patched, but they said the reason I had a flat was because the previous patch had blown - WAIT, WHAT? I just got the tire and never needed to have it patched? This was supposed to be a new tire. \\nI took the tire over to Flynn\'s and they told me that someone punctured my tire, then tried to patch it. So there are resentful tire slashers? I find that very unlikely. After arguing with the guy and telling him that his logic was far fetched he said he\'d give me a new tire \\"this time\\". \\nI will never go back to Flynn\'s b/c of the way this guy treated me and the simple fact that they gave me a used tire!'
}
```

#### 数据字段

- 'text': 评论文本使用双引号（"）转义，任何内部双引号都通过2个双引号（""）转义。换行符使用反斜杠后跟一个 "n" 字符转义，即 "\n"。
- 'label': 对应于评论的分数（介于1和5之间）。

#### 数据拆分

Yelp评论完整星级数据集是通过随机选取每个1到5星评论的130,000个训练样本和10,000个测试样本构建的。总共有650,000个训练样本和50,000个测试样本。

## 下载数据集

In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [2]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [3]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [4]:
show_random_elements(dataset["train"])

,label,text
0,2 star,Nice ambiance but pasta was dry and cold
1,2 star,"This review is for a lunch seating. \n\nWe sat down, ordered, ate and nothing came back up. That's about the only nice thing I can say about the food. \n\nService was very friendly and it's got decent ambience, especially on the patio. \n\nIf you must eat here, unless you enjoy government cheese melted onto soggy cardboard, I suggest you avoid the pizzas."
2,5 stars,"I just bought there honey at Costco, and it's delicious! I love the flavor of it, especially on plain yogurt. I will definitely purchase again. Thanks Crockett Honey!"
3,4 stars,"I recently went from mild vegetarian to full on vegan. I cook a lot at home because its not easy finding restaurants to eat at, especially in Las Vegas, the steakhouse capital of the world. I decided I needed to get out for Valentine's Day and chose to go Indian. I came across Tamba's amazing website and easily made up my mind. After making the reservation I read a few revues on Google and they worried me a bit, but I wasn't going to be shaken. I'm so glad I stayed with it, I loved this place. Staff was very friendly and the food was delicious! We tried the Bhel Puri for an appetizer, based off of the waiters recommendation, great choice. Then for entrees we went with the Aloo Gobi & Chana Masala. Both were amazing and complimented each other nicely, great for sharing. She went with a glass of house Chard, which turned out to be Mondavi(Nice!) and I tried one of the many Indian Imported beers. Hayward 5000 was the beer I went with and was perfect. Gotta find that at Lee's Discount Liquor, hope they have it. \n\nOne side note, when they ask you for bread, it's not included. That was awkward. Also, parking is a bit rough.\n\nOver all this was probably one of the least expensive Vday dinners I've ever had and also one of the most satisfying."
4,2 star,Run of the mill cheap Chinese fast food and other poor quality fast food grade food.
5,2 star,"I was really disappointed by my experience at craftsteak. Ate here with 5 of my dearest oldest friends for my bachelorette weekend. Huge fan of top chef, and when I discovered tom collichio owned this place I just had to try it for our big saturday night meal. We are all big foodies. And no, I was not wearing a veil and tiara or any blinking jewelry. We are all in our late 20s early 30s and were well behaved, at least til the end of the night when we had consumed several martinis and bottles of vino. We may have been laughing loudly but we never got rowdy...and we were practically in a PDR for chrissake so I could've mooned the table next to me and only about a dozen or so people might see. Not that I would. Just sayin, we were hidden away. All that being said...service was freaking awful. Our server was an older gentleman, and he was downright rude. If 18% gratuity had not been tacked on, I honestly might have gone 15% (which I NEVER do being a former server and bartender myself..I have maybe done this once in my life!). I ordered the surf & turf - a splurge - but usually one of my favorite things to eat. The filet was a tad undercooked for medium rare, but I kinda like it in between rare and med rare so it was fine by me. One of my friends couldn't handle it! The lobster was so rubbery and chewy and coldish and flavorless...did it come out of the fridge? Side dishes bored me (and everyone). My opening salad was good - but only because we shared a ton of starters and I mixed the mozzarella salad with the tomato salad to make my own caprese. I think one of my friends enjoyed her short ribs. Nobody raved about anything. Except maybe the hot sticky buns that come out in the beginning. Truly sad we did not enjoy it! Should've eaten elsewhere. Will not return."
6,5 stars,"I had my Bar Mitzvah here, and the event was great. The whitewater rafting is fun for all ages, and we've gone back several times. There are plenty of other things to do, such as rock climbing, a ropes course, and a 

## 预处理数据

下载数据集到本地后，使用 Tokenizer 来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。

Datasets 的 `map` 方法，支持一次性在整个数据集上应用预处理函数。

下面使用填充到最大长度的策略，处理整个数据集：

In [10]:
modal_name = "google-bert/bert-large-uncased"

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(modal_name)

In [14]:
text = dataset["train"]["text"][0]
print(text)
token = tokenizer.tokenize(text)
print(token)
tokenizer.encode_plus(text)

dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.
['dr', '.', 'goldberg', 'offers', 'everything', 'i', 'look', 'for', 'in', 'a', 'general', 'practitioner', '.', 'he', "'", 's', 'nice', 'and', 'easy', 'to', 'talk', 'to', 'without', 'being', 'patron', '##izing', ';', 'he', "'", 's', 'always', 'on', 'time', 'in', 'seeing', 'his', 'patients', ';', 'he', "'", 's', 'affiliated', 'with', 'a', 'top', '-', 'notch', 'hospital', '(', 'nyu', ')', 'which', 'my', 'parents', 'have', 'explained', 'to', 'me', 'is', 'very', 'im

{'input_ids': [101, 2852, 1012, 18522, 4107, 2673, 1045, 2298, 2005, 1999, 1037, 2236, 18742, 1012, 2002, 1005, 1055, 3835, 1998, 3733, 2000, 2831, 2000, 2302, 2108, 9161, 6026, 1025, 2002, 1005, 1055, 2467, 2006, 2051, 1999, 3773, 2010, 5022, 1025, 2002, 1005, 1055, 6989, 2007, 1037, 2327, 1011, 18624, 2902, 1006, 27935, 1007, 2029, 2026, 3008, 2031, 4541, 2000, 2033, 2003, 2200, 2590, 1999, 2553, 2242, 6433, 1998, 2017, 2342, 5970, 1025, 1998, 2017, 2064, 2131, 6523, 7941, 2015, 2000, 2156, 15744, 2302, 2383, 2000, 2156, 2032, 2034, 1012, 2428, 1010, 2054, 2062, 2079, 2017, 2342, 1029, 1045, 1005, 1049, 3564, 2182, 2667, 2000, 2228, 1997, 2151, 10821, 1045, 2031, 2055, 2032, 1010, 2021, 1045, 1005, 1049, 2428, 5059, 1037, 8744, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=600)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

### 数据抽样

使用 1000 个数据样本，在 BERT 上演示小规模训练（基于 Pytorch Trainer）

`shuffle()`函数会随机重新排列列的值。如果您希望对用于洗牌数据集的算法有更多控制，可以在此函数中指定generator参数来使用不同的numpy.random.Generator。

In [ ]:
train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

## 微调训练配置

### 加载 BERT 模型

警告通知我们正在丢弃一些权重（`vocab_transform` 和 `vocab_layer_norm` 层），并随机初始化其他一些权重（`pre_classifier` 和 `classifier` 层）。在微调模型情况下是绝对正常的，因为我们正在删除用于预训练模型的掩码语言建模任务的头部，并用一个新的头部替换它，对于这个新头部，我们没有预训练的权重，所以库会警告我们在用它进行推理之前应该对这个模型进行微调，而这正是我们要做的事情。

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(modal_name, num_labels=5)
model.config.pad_token_id = model.config.eos_token_id

### 训练超参数（TrainingArguments）

完整配置参数与默认值：https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments

源代码定义：https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/training_args.py#L161

**最重要配置：模型权重保存路径(output_dir)**

In [ ]:
model_dir = "models/gpt2-finetune-yelp"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
from transformers import TrainingArguments
num_epochs = 2
batch_size = 15
warmup_steps = int(0.1 * (len(train_dataset) * num_epochs // batch_size))  # 10% of total steps

training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="steps",
                                  per_device_train_batch_size=batch_size,
                                  num_train_epochs=num_epochs,
                                  logging_steps=4000,
                                  weight_decay=0.01,
                                  warmup_steps=warmup_steps)

In [ ]:
# 完整的超参数配置
print(training_args)

### 训练过程中的指标评估（Evaluate)

**[Hugging Face Evaluate 库](https://huggingface.co/docs/evaluate/index)** 支持使用一行代码，获得数十种不同领域（自然语言处理、计算机视觉、强化学习等）的评估方法。 当前支持 **完整评估指标：https://huggingface.co/evaluate-metric**

训练器（Trainer）在训练过程中不会自动评估模型性能。因此，我们需要向训练器传递一个函数来计算和报告指标。 

Evaluate库提供了一个简单的准确率函数，您可以使用`evaluate.load`函数加载

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


接着，调用 `compute` 函数来计算预测的准确率。

在将预测传递给 compute 函数之前，我们需要将 logits 转换为预测值（**所有Transformers 模型都返回 logits**）。

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## 开始训练

### 实例化训练器（Trainer）

`kernel version` 版本问题：暂不影响本示例代码运行

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64)

In [ ]:
trainer.evaluate(small_test_dataset)

### 保存模型和训练状态

- 使用 `trainer.save_model` 方法保存模型，后续可以通过 from_pretrained() 方法重新加载
- 使用 `trainer.save_state` 方法保存训练状态

In [ ]:
trainer.save_model(model_dir)

In [ ]:
trainer.save_state()